## Win/Loss Rating Model Prediction

Load the model and make predictions

In [1]:
import requests
import pandas as pd
import numpy as np
import pymc3 as pm
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm
from spcl_case import *
plt.style.use('fivethirtyeight')
%matplotlib inline

### Get a list of all CS Games

In [2]:
r = requests.get('https://thunderpick.com/api/matches')
games = pd.DataFrame(r.json()['data'])
games = games[games.gameId == 6].sort_values('championship')

bet_games = []
for i,v in games.iterrows():
    if((v['isTournament'] == False )& (v['canWager'] == True)):
        ratio = v['matchBet']['buckets'][0]['amount']/v['matchBet']['buckets'][1]['amount']
        odds = (ratio**-1+1, ratio+1)
        wr = (odds[1]/np.sum(odds)*100., odds[0]/np.sum(odds)*100.)
        det = requests.get('https://thunderpick.com/api/matches/'+str(v['id'])).json()['data']
        print('Date: %s  |  Event: %s  | (BO%s) %s vs. %s  |  (%.1f:%.1f) | Total Coins: %i' % 
              (v['startTime'][:10], v['championship'], det['bestOfMaps'], v['matchBet']['buckets'][0]['label'], 
               v['matchBet']['buckets'][1]['label'], wr[0], wr[1], v['matchBet']['amount']))
        bet_games.append({'1': v['matchBet']['buckets'][0]['label'], '2': v['matchBet']['buckets'][1]['label'], 'bo': det['bestOfMaps'], 'o1': odds[0], 'o2': odds[1], 'wr': wr[0]})
bet_games = pd.DataFrame(bet_games)

Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) EnVyUs vs. fnatic  |  (52.0:48.0) | Total Coins: 2000
Date: 2017-10-26  |  Event: ECS Season 4 Development League  | (BO1) SK vs. Liquid  |  (62.8:37.2) | Total Coins: 2150
Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) Immortals vs. Luminosity  |  (56.0:44.0) | Total Coins: 2144
Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) Luminosity vs. Cloud9  |  (39.6:60.4) | Total Coins: 10100
Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) Cloud9 vs. Luminosity  |  (64.5:35.5) | Total Coins: 11258
Date: 2017-10-26  |  Event: ECS Season 4 Development League  | (BO1) Immortals vs. Ghost  |  (61.0:39.0) | Total Coins: 2050
Date: 2017-10-24  |  Event: ECS Season 4 Development League  | (BO1) GODSENT vs. fnatic  |  (39.0:61.0) | Total Coins: 2050
Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) fnatic vs. EnVyUs  |  (45.0:55.0) | Total Coins: 2000


## Load Ratings Model

In [27]:
TEAM_SET = 'mdleu'

teams = np.load('saved_model/'+TEAM_SET+'/teams.npy')
maps = np.load('saved_model/'+TEAM_SET+'/maps.npy')
filt = np.chararray.lower(np.load('saved_model/'+TEAM_SET+'/filter_teams.npy'))
h_teams = pd.read_csv('hltv_csv/teams.csv').set_index('ID').loc[teams]
h_teams = fix_teams(h_teams)
h_teams.Name = h_teams.Name.str.lower()
h_teams_filt = h_teams[h_teams.Name.isin(filt)]

rating_model = prep_pymc_model(len(teams), len(maps))
trace = pm.backends.text.load('saved_model/'+TEAM_SET+'/trace', model=rating_model)

## Ban/Pick Predictions

In [11]:
h_bp = pd.read_csv('hltv_csv/picksAndBans.csv').set_index('Match ID')
h_matches = pd.read_csv('hltv_csv/matchResults.csv').set_index('Match ID')
h_matches['Date'] = pd.to_datetime(h_matches['Date'])
h_matches = h_matches[h_matches['Date'] >= dt.datetime(2017,1,1)]
h_bp = h_bp.join(h_matches[['Date']], how='left')
h_bp['Date'] = pd.to_datetime(h_bp['Date'])
h_bp = h_bp[h_bp['Date'] >= dt.datetime(2017,1,1)]

In [12]:
def model_mp(train, t1, t2):
    tab = train[train['Team'].isin([t1, t2])].groupby(['Team', ' Pick Type', 'Map'])['Date'].count().unstack([' Pick Type', 'Team']).fillna(0)
    return (tab/tab.sum(axis=0)).mean(level=0,axis=1)# get average

def model_played(train, t1, t2):
    a = train[train['Team 1 ID'].isin([t1,t2])].groupby(['Team 1 ID', 'Map'])['Date'].count()
    b = train[train['Team 2 ID'].isin([t1,t2])].groupby(['Team 2 ID', 'Map'])['Date'].count()
    c = pd.DataFrame([a,b], index=['a','b']).T.fillna(0)
    c = (c['a']+c['b']).unstack(level=0).fillna(0)
    return (c/c.sum()).mean(axis=1)

def predict_map(func, data, t1, t2):
    res = func(data, t1, t2)
    return res.loc[res.index != 'Default'].sort_values(ascending=False)

# Bet Predictions

In [29]:
money = 4500.
bet_games['1'] = bet_games['1'].str.lower().replace('ex-Denial', 'Denial')
bet_games['2'] = bet_games['2'].str.lower().replace('ex-Denial', 'Denial')
matches = bet_games[bet_games['1'].isin(filt) & bet_games['2'].isin(filt)].drop_duplicates()
def sig(x):
    return 1 / (1 + np.exp(-x))
def abs_norm_interval(start,end,loc,scale):
    return (norm.cdf(end,loc,scale) - norm.cdf(start,loc,scale)) + (norm.cdf(-1*start,loc,scale) - norm.cdf(-1*end,loc,scale))

t_rating = trace['rating']
t_map_rating = trace['rating | map']
t_alpha = trace['alpha']
for i,v in matches.iterrows():
    t1_id = h_teams_filt[h_teams_filt.Name == v['1']].index[0]; t1_ind = np.where(teams == t1_id)[0][0];
    t2_id = h_teams_filt[h_teams_filt.Name == v['2']].index[0]; t2_ind = np.where(teams == t2_id)[0][0];
    trace_1 = t_rating[:,t1_ind]; trace_2 = t_rating[:,t2_ind]
    mr_1 = trace_1.mean(); mr_2 = trace_2.mean();
    diff = trace_1-trace_2
    p_wl = sig(diff)
    wr_25 = np.percentile(p_wl, 25); wr_75 = np.percentile(p_wl, 75)
    kelly_pct_1 = ((v['o1']*np.percentile(p_wl, 45)-(1.-np.percentile(p_wl, 45)))/v['o1'])*0.1
    kelly_pct_2 = ((v['o2']*(1.-np.percentile(p_wl, 45))-(np.percentile(p_wl, 45)))/v['o2'])*0.1
    print('%s (%.3f) vs %s (%.3f) - I:%.2f%% | P:%.2f%% - %.2f%%  -  K: %.1f%% (%i) - %.1f%% (%i)' % 
          (v['1'], mr_1, v['2'], mr_2, v['wr'], wr_25*100, wr_75*100, kelly_pct_1*100., 
           kelly_pct_1*money, kelly_pct_2*100., kelly_pct_2*money))

kinguin (1.584) vs igame.com (1.104) - I:67.73% | P:50.71% - 71.74%  -  K: 3.2% (144) - 2.1% (95)
extatus (1.474) vs pride (0.835) - I:68.76% | P:54.92% - 74.78%  -  K: 3.8% (172) - 1.7% (75)
mk (0.951) vs japaleno (1.166) - I:34.88% | P:31.58% - 58.52%  -  K: 2.2% (97) - 3.1% (138)
extatus (1.474) vs invictus aquilas (-0.205) - I:71.30% | P:76.45% - 89.83%  -  K: 7.1% (318) - -0.7% (-30)
singularity (1.499) vs endpoint (-0.660) - I:79.00% | P:83.93% - 93.51%  -  K: 8.0% (359) - -0.7% (-33)
invictus aquilas (-0.205) vs singularity (1.499) - I:27.00% | P:9.78% - 23.36%  -  K: -0.9% (-39) - 7.5% (338)
1337huania (0.036) vs space soldiers (2.429) - I:16.63% | P:4.96% - 13.87%  -  K: -0.8% (-35) - 8.6% (387)
vitalis (0.930) vs singularity (1.499) - I:37.00% | P:25.26% - 49.07%  -  K: 1.0% (43) - 4.4% (200)
pride (0.835) vs aaa (0.817) - I:55.61% | P:31.64% - 69.35%  -  K: 1.7% (76) - 3.3% (147)
ago (1.863) vs virtus.pro (1.743) - I:52.20% | P:41.16% - 64.52%  -  K: 2.5% (112) - 2.5% (112)


In [21]:
filt

array(['Virtus.pro', 'FlipSid3', 'eXtatus', 'AGO', 'Fragsters', 'Gambit',
       'PRIDE', '1337HUANIA', 'VITALIS', 'Epsilon', 'CHAOS', 'Crowns',
       'MK', 'Japaleno', 'Not Academy', 'aAa', 'Space Soldiers',
       'Singularity', 'Nexus', 'Invictus Aquilas', 'Spirit', 'Kinguin',
       'Seed', 'Endpoint', 'iGame.com', 'TEAM5', 'ALTERNATE aTTaX'],
      dtype='<U16')

In [30]:
PRINT_RD_DIFF = False
for i,v in matches.iterrows():
    t1_id = h_teams_filt[h_teams_filt.Name == v['1']].index[0]; t1_ind = np.where(teams == t1_id)[0][0];
    t2_id = h_teams_filt[h_teams_filt.Name == v['2']].index[0]; t2_ind = np.where(teams == t2_id)[0][0];
    print('---------- %s vs %s ---------------------------------' % (v['1'], v['2']))
    pred_maps = predict_map(model_played, h_matches, t1_id, t2_id)
    pred_maps = pred_maps/pred_maps.sum()
    for m,s in pred_maps.iteritems():
        m_ind = np.where(maps == m)[0][0]
        trace_1 = t_map_rating[:,m_ind,t1_ind]; trace_2 = t_map_rating[:,m_ind,t2_ind]
        mr_1 = trace_1.mean(); mr_2 = trace_2.mean();
        diff = trace_1-trace_2
        p_wl = sig(diff)
        wr_25 = np.percentile(p_wl, 25); wr_75 = np.percentile(p_wl, 75)
        kappa = 32*sig(t_alpha*diff)-16.
        kelly_pct_1 = ((v['o1']*np.percentile(p_wl, 45)-(1.-np.percentile(p_wl, 45)))/v['o1'])*0.1
        kelly_pct_2 = ((v['o2']*(1.-np.percentile(p_wl, 45))-(np.percentile(p_wl, 45)))/v['o2'])*0.1
        print('    Map: %s (%.2f)  -  %s (%.3f) vs %s (%.3f) - I:%.2f%% | P:%.2f%% - %.2f%%  -  K: %.1f%% (%i) - %.1f%% (%i)' % 
             (m, s*100., v['1'], mr_1, v['2'], mr_2, v['wr'], wr_25*100, wr_75*100, kelly_pct_1*100., 
               kelly_pct_1*money, kelly_pct_2*100., kelly_pct_2*money))
        
        if(PRINT_RD_DIFF):
            p_sc = [abs_norm_interval(x[0],x[1],kappa,trace['sigma'][:,m_ind]) for x in [[1.5,3.5],[3.5,5.5],[5.5,7.5],[7.5,9.5],[9.5,16]]]
            for i,sd in enumerate(['2 - 3 Rounds', '4 - 5 rounds', '6 - 7 rounds', '8 - 9 rounds', '10 rounds or more']):
                sc_25 = np.percentile(p_sc[i], 25); sc_75 = np.percentile(p_sc[i], 75)
                print('      %s : %.2f%% - %.2f%%' % (sd, sc_25*100, sc_75*100))

---------- kinguin vs igame.com ---------------------------------
    Map: Train (20.45)  -  kinguin (1.930) vs igame.com (1.988) - I:67.73% | P:43.04% - 54.17%  -  K: 1.2% (53) - 3.7% (167)
    Map: Overpass (17.51)  -  kinguin (1.674) vs igame.com (1.021) - I:67.73% | P:59.64% - 71.44%  -  K: 4.1% (182) - 1.5% (65)
    Map: Mirage (15.37)  -  kinguin (2.070) vs igame.com (0.185) - I:67.73% | P:83.34% - 89.61%  -  K: 7.7% (345) - -1.4% (-63)
    Map: Cobblestone (12.25)  -  kinguin (1.350) vs igame.com (-0.230) - I:67.73% | P:77.98% - 86.92%  -  K: 7.0% (313) - -0.8% (-37)
    Map: Inferno (11.47)  -  kinguin (1.872) vs igame.com (1.756) - I:67.73% | P:44.85% - 61.00%  -  K: 1.8% (81) - 3.2% (145)
    Map: Cache (10.23)  -  kinguin (0.445) vs igame.com (1.120) - I:67.73% | P:25.36% - 43.04%  -  K: -1.4% (-62) - 5.7% (258)
    Map: Nuke (9.88)  -  kinguin (2.532) vs igame.com (1.423) - I:67.73% | P:66.72% - 81.99%  -  K: 5.6% (253) - 0.2% (9)
    Map: Dust2 (2.85)  -  kinguin (1.913) v

    Map: Cache (13.18)  -  vitalis (2.237) vs singularity (0.041) - I:37.00% | P:85.11% - 93.39%  -  K: 8.5% (383) - -4.5% (-204)
    Map: Cobblestone (9.70)  -  vitalis (-0.637) vs singularity (1.850) - I:37.00% | P:4.29% - 13.62%  -  K: -2.8% (-123) - 8.9% (399)
    Map: Nuke (8.41)  -  vitalis (0.928) vs singularity (2.550) - I:37.00% | P:6.74% - 35.22%  -  K: -1.8% (-79) - 7.7% (346)
---------- pride vs aaa ---------------------------------
    Map: Inferno (37.39)  -  pride (0.463) vs aaa (2.485) - I:55.61% | P:5.88% - 22.52%  -  K: -3.9% (-175) - 8.5% (380)
    Map: Mirage (28.19)  -  pride (2.265) vs aaa (-0.300) - I:55.61% | P:84.47% - 96.78%  -  K: 8.7% (390) - -3.2% (-144)
    Map: Train (10.71)  -  pride (1.979) vs aaa (0.826) - I:55.61% | P:50.24% - 90.75%  -  K: 5.6% (253) - -0.4% (-17)
    Map: Cache (8.44)  -  pride (0.936) vs aaa (0.817) - I:55.61% | P:26.03% - 78.53%  -  K: 1.9% (85) - 3.1% (138)
    Map: Nuke (7.47)  -  pride (1.372) vs aaa (0.827) - I:55.61% | P:35.7

In [36]:
np.inner(np.ones((8,25)), np.ones((8,25))).shape

(8, 8)

In [40]:
np.ones((8,25)).flatten().shape

(200,)

In [41]:
np.zeros((8000, 200))

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [ ]:
plt.ylim(0,1.2)
sns.kdeplot(trace_1, shade=True, alpha=0.65, legend=True, label=v['1'])
sns.kdeplot(trace_2, shade=True, alpha=0.65, legend=True, label=v['2'])

In [ ]:
h_bp.groupby('Match ID').first().count()

In [ ]:
h_bp